In [1]:
import logging

from automata.cli.commands import reconfigure_logging
from automata.config.base import AgentConfigName
from automata.config.openai_agent import OpenAIAutomataAgentConfigBuilder
from automata.agent.providers import OpenAIAutomataAgent
from automata.singletons.dependency_factory import dependency_factory
from automata.singletons.py_module_loader import py_module_loader
from automata.tools.factory import AgentToolFactory

logger = logging.getLogger(__name__)
reconfigure_logging("DEBUG")

py_module_loader.initialize()

INFO:automata.singletons.py_module_loader:Loading modules with root path: /Users/ocolegrove/automata_fresh_2/automata/core/../.. and py path: /Users/ocolegrove/automata_fresh_2/automata/core/../../automata


In [2]:
# Construct the set of all dependencies that will be used to build the tools
toolkit_list = ["context-oracle"]
tool_dependencies = dependency_factory.build_dependencies_for_tools(toolkit_list)

INFO:automata.singletons.dependency_factory:Building dependencies for toolkit_list ['context-oracle']...
INFO:automata.singletons.dependency_factory:Building symbol_search...
INFO:automata.singletons.dependency_factory:Creating dependency symbol_search
INFO:automata.singletons.dependency_factory:Creating dependency symbol_graph
INFO:automata.singletons.dependency_factory:Creating dependency symbol_code_embedding_handler
INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
INFO:clickhouse_connect.driver.ctypes:Successfully imported ClickHouse Connect C data optimizations
DEBUG:clickhouse_connect.driver.ctypes:Successfully import ClickHouse Connect C/Numpy optimizations
INFO:clickhouse_connect.json_impl:Using python library for writing JSON byte strings
INFO:chromadb.db.duckdb:loaded in 963 embeddings
INFO:chromadb.db.duckdb:loaded in 4 collections
DEBUG:chromadb.config:Starting component System
DEBUG:chromadb.config

In [3]:
# Build the tools
tools = AgentToolFactory.build_tools(toolkit_list, **tool_dependencies)

In [4]:
# Build the agent config
config_name = AgentConfigName("automata-main")

agent_config = (
    OpenAIAutomataAgentConfigBuilder.from_name(config_name)
    .with_tools(tools)
    .with_model("gpt-4")
    .with_max_iterations(2)
    .build()
)

INFO:automata.singletons.dependency_factory:Creating dependency symbol_rank
INFO:automata.singletons.dependency_factory:Creating dependency subgraph
INFO:automata.symbol.graph.navigator:Pre-computing bounding boxes for all rankable symbols
INFO:automata.symbol.graph.navigator:Finished pre-computing bounding boxes for all rankable symbols in 1.0803437232971191 seconds
INFO:automata.symbol.graph.symbol_graph:Building the rankable symbol subgraph...
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 633/633 [00:01<00:00, 430.51it/s]
INFO:automata.symbol.graph.symbol_graph:Built the rankable symbol subgraph


In [11]:
# Initialize the agent
# instructions = "Explain how OpenAI is used by the codebase. Note in particularly how it is used in embeddings and how agents consume it"
instructions = "Provide a comprehensive explanation of how SymbolRank and SymbolSearch work. Return your answer as a valid JSON string."
import textwrap
instructions = textwrap.dedent('''
Your task is to solve the following Github Issue

Title:
Specify list of supported symbols for document generation, add override flag to `EmbeddingHandler`

Body:
The goal is to modify the `run_doc_embedding` script to accept an arbitrary list of new symbols to regenerate [preferably in some human readable form that is parsed]. These symbols should then overwrite existing results in the database with new documentation if necessary. To implement the second step a flag will need to be added to the EmbeddingHandler.


1. Understand the Project: Review the Automata project to get a clear understanding of its goals and how it operates. This includes understanding how the document generation pipeline works, the role of indices and embeddings, and how they interplay to allow for efficient code and documentation generation.

2. Setup the Environment: Clone the Automata project into the "../repo_store/automata" directory relative to your local working directory.

    Code: `git clone git@github.com:emrgnt-cmplxty/Automata.git ../repo_store/automata`

3. Generate New Indices: Once you have cloned the Automata project, navigate to the "scripts" directory and run the `generate_indices.sh` script to generate new indices.

    Code: 
    ```bash
    cd scripts
    ./generate_indices.sh
    ```

    You can verify the creation of the indices by navigating to the `automata-embedding-data` directory and running `git status`.

    Code: 
    ```bash
    cd automata-embedding-data
    git status
    ```
4. Refresh Code Embeddings: From your main directory, run the `run-code-embedding` command. This will refresh the embeddings in the database, rolling the commit hash forward where the symbol source code hasn't changed and recalculating the index where necessary.

    Code: `poetry run automata run-code-embedding`

5. Run Document Embedding: Next, run the `run-doc-embedding` command locally to see it in action.

    Code: `poetry run automata run-doc-embedding`

    This command generates new docs for newly added symbols and moves forward with the symbols that are in the index and map onto the database. This is a crucial part of the pipeline which generates the Automata docs.

6. Understand the Codebase: As you carry out these steps, ensure to take mental notes to understand the workings of the codebase. Insert print statements and other debug aids to get a sense of what's happening in the pipeline.

7. Modify `run_doc_embedding`: Once you are confident with your understanding of the pipeline, your goal is to modify `run_doc_embedding` to accept a list of new symbols to regenerate, and overwrite existing results in the database with new documentation if necessary.

8. Seek Help: If you encounter any issues or have questions, don't hesitate to ask for help. Once you have successfully carried out these tasks, check in for the next steps.

Remember, understanding the pipeline and how everything fits together is the key to this task. 

Begin now.
''')
agent = OpenAIAutomataAgent(instructions, config=agent_config)

DEBUG:automata.agent.providers:Setting up agent with tools = [OpenAITool(function=<bound method ContextOracleToolkitBuilder._get_context of <automata.tools.builders.context_oracle.ContextOracleOpenAIToolkitBuilder object at 0x106d66f10>>, name='context-oracle', description="This tool utilizes the EmbeddingSimilarityCalculator and SymbolSearch to provide context for a given query by computing semantic similarity between the query and all available symbols' documentation and code. The symbol with the highest combined similarity score is identified, with its source code and documentation summary forming the primary context. Additionally, if enabled, the documentation summaries of related symbols (those next most similar to the query) are included.", coroutine=None, properties={'query': {'type': 'string', 'description': 'The query string to search for.'}, 'max_additional_related_symbols': {'type': 'integer', 'description': 'The maximum number of additional related symbols to return documen

DEBUG:automata.agent.providers:
------------------------------------------------------------
Session ID: 8728958f-7b7b-4472-bfcd-83f795c7f58d
------------------------------------------------------------




In [12]:
# Run the agent
result = agent.run()

DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 3129 tokens were consumed prior to completion generation.


In order to modify the `run-doc-embedding` script to accept an arbitrary list of new symbols and overwrite existing results, we first must understand how the documentation generation pipeline works in the Automata codebase. 

Particularly, we are interested in the `EmbeddingHandler` and its role in the pipeline, as it appears to be responsible for creating and updating the embeddings used for code and documentation generation. As the next step, let's look at the current implementation of the `run-doc-embedding` script and the `EmbeddingHandler`. 

I'm going to call `context_oracle` function to get the current implementation of the script and the handler.Function Call:
context-oracle

Arguments:
{
  "query": "run_doc_embedding"
}

 

DEBUG:automata.llm.providers.openai:Approximately 3302 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:



class SymbolDocEmbeddingBuilder(EmbeddingBuilder):
    'Builds `Symbol` documentation embeddings.'

    def __init__(self, embedding_provider: EmbeddingVectorProvider, completion_provider: LLMChatCompletionProvider, symbol_search: SymbolSearch, handler: PyContextHandler) -> None:
        super().__init__(embedding_provider)
        self.completion_provider = completion_provider
        self.symbol_search = symbol_search
        self.handler = handler

    def build(self, source_code: str, symbol: Symbol) -> SymbolDocEmbedding:
        '\n        Build the embedding for a symbol\'s documentation.\n        Example Document Output:\n        ===========\n        AgentConfi

DEBUG:automata.llm.providers.openai:Approximately 7457 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
Latest Assistant Message -- 

DEBUG:automata.llm.providers.openai:Approximately 7457 tokens were consumed prior to completion generation.


Function Call:
call_termination

Arguments:
{"result": "To solve the issue, we need to refactor the `run_doc_embedding` script and the `EmbeddingHandler` class as follows:

1. **run_doc_embedding script:**
The script needs to accept an input list (array) of symbols that need to be regenerated. It also needs to process this list (in addition to whatever it already does) and only regenerate these specified symbols. An example of this could be having the script accept a command line argument that points to a json or txt file containing a list of symbols, which it then parses.

2. **EmbeddingHandler class:**
Add a flag (boolean) property named something like 'overwrite_existing'. When this flag is set to true, if an embedding for the symbol already exists in the database, it should be overwritten instead of skipped. The
default value for this flag should be 'False' so that the default behavior remains unchanged.

Code changes will involve adding the flag in the `EmbeddingHandler`. It would

DEBUG:automata.llm.providers.openai:Approximately 7969 tokens were after adding the latest message.
DEBUG:root:
------------------------------------------------------------------------------------------------------------------------
DEBUG:automata.agent.providers:Latest User Message -- 
user:
content=Execution Result:

To solve the issue, we need to refactor the `run_doc_embedding` script and the `EmbeddingHandler` class as follows:

1. **run_doc_embedding script:**
The script needs to accept an input list (array) of symbols that need to be regenerated. It also needs to process this list (in addition to whatever it already does) and only regenerate these specified symbols. An example of this could be having the script accept a command line argument that points to a json or txt file containing a list of symbols, which it then parses.

2. **EmbeddingHandler class:**
Add a flag (boolean) property named something like 'overwrite_existing'. When this flag is set to true, if an embedding for

In [7]:
# Print the result
print(f"Result:\n{result}")

Result:
Execution Result:

{"SymbolRank": {"description": "SymbolRank class applies the PageRank algorithm on a graph to rank symbols such as methods and classes based on their semantic context and structural relationships within a software. Symbols are the classes, methods or other elements in a code corpus. A SymbolGraph is constructed where each symbol forms a node and dependencies between symbols form edges. This SymbolGraph maps structural information from the codebase and helps explore symbol dependencies, relationships and hierarchy. Finally, a prepared similarity dictionary between symbols is used in combination with the SymbolGraph to compute their SymbolRanks. This is performed using an iterative computation analogous to Google's PageRank algorithm, considering symbols' similarity scores and their connectivity within the graph.", "methods": {"__init__": "Initializes a SymbolRank instance with a given graph and a SymbolRankConfig. If config is not provided, a default SymbolRan

In [ ]:
result